In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
yearly_interest_rate = widgets.BoundedFloatText(min = 2, max = 15, value = 4, 
                                                step = 0.01, description = 'Interest Rate:')

down_payment = widgets.BoundedFloatText(min = 0, max = 9999999, value = 20000, 
                                                step = 1, description = 'Down Payment:')

house_price = widgets.BoundedFloatText(min = 10000, max = 10000000, value = 100000, 
                                                step = 1, description = 'House Price:')

mortgage_term = widgets.BoundedFloatText(min = 5, max = 50, value = 30, 
                                                step = 1, description = 'Mortgage Term::')


calculator_output = widgets.Output()
def calculator(houseprice, downpayment, interest, mortgageterm):
    calculator_output.clear_output()
    monthlyint =  interest/1200
    loan = houseprice - downpayment
    totalmonth = mortgageterm*12
    monthlypmt = (loan*monthlyint) / (1 - ((1+monthlyint)**(-totalmonth)))
    with calculator_output:
        print('The money is {}:'.format(monthlypmt))


def money2pay_1(houseprice, downpayment, interest, mortgageterm, month):
    monthlyint =  interest/1200
    loan = houseprice - downpayment
    totalmonth = mortgageterm*12
    balance = loan * ((1+monthlyint)**totalmonth - (1+monthlyint)**month)/((1+monthlyint)**totalmonth - 1)
    return balance
       
money2pay_output_1 = widgets.Output()
plot_output_1 = widgets.Output()

def Money2Pay_1(houseprice, downpayment, interest, mortgageterm):
    money2pay_output_1.clear_output()
    plot_output_1.clear_output()
    Money_2_Pay = []
    Interest_2_Pay = []
    Principal_2_Pay = []
    monthlyint =  interest/1200
    loan = houseprice - downpayment
    totalmonth = mortgageterm*12
    monthlypmt = (loan*monthlyint) / (1 - ((1+monthlyint)**(-totalmonth)))
    for i in range(0, int(totalmonth+1)):
        Money_2_Pay.append(money2pay_1(houseprice, downpayment, interest, mortgageterm, i))
        Interest_2_Pay.append(money2pay_1(houseprice, downpayment, interest, mortgageterm, i) *  monthlyint)
        MoneyDue = pd.DataFrame([Money_2_Pay,Interest_2_Pay]).T
        MoneyDue.columns = ['Loan To Pay', 'Interest To Pay']
        MoneyDue['Principal To Pay'] = monthlypmt - MoneyDue['Interest To Pay']
        MoneyDue['Monthly Payment'] = pd.DataFrame([monthlypmt]*int(totalmonth))
        MoneyDue['Month'] = pd.DataFrame(range(1, int(totalmonth+1)))
        MoneyDue = MoneyDue.drop(MoneyDue.tail(1).index)
    with money2pay_output_1:
        display(MoneyDue)
    with plot_output_1:
        sns.lineplot(x='Month', y='value', hue='variable', data=pd.melt(MoneyDue.iloc[:, 1:], ['Month']))
        plt.show()



def house_price_eventhandler(change):
    calculator(change.new, down_payment.value, yearly_interest_rate.value, mortgage_term.value)
    Money2Pay_1(change.new, down_payment.value, yearly_interest_rate.value, mortgage_term.value)
          
def down_payment_eventhandler(change):
    calculator(house_price.value, change.new, yearly_interest_rate.value, mortgage_term.value)
    Money2Pay_1(house_price.value, change.new, yearly_interest_rate.value, mortgage_term.value)
    
def yearly_interest_eventhandler(change):
    calculator(house_price.value, down_payment.value, change.new, mortgage_term.value)
    Money2Pay_1(house_price.value, down_payment.value, change.new, mortgage_term.value)
def mortgage_term_eventhandler(change):
    calculator(house_price.value, down_payment.value, yearly_interest_rate.value, change.new)
    Money2Pay_1(house_price.value, down_payment.value, yearly_interest_rate.value, change.new)
    
house_price.observe(house_price_eventhandler, names='value')
down_payment.observe(down_payment_eventhandler, names='value')
yearly_interest_rate.observe(yearly_interest_eventhandler, names='value')
mortgage_term.observe(mortgage_term_eventhandler, names='value')
    
    
item_layout = widgets.Layout(margin='0 0 50px 0')
input_widgets = widgets.HBox([house_price, down_payment, yearly_interest_rate, mortgage_term], layout = item_layout)
tab = widgets.Tab([money2pay_output_1, plot_output_1], layout = item_layout)
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'Amortization BreakDown')
dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)

In [3]:
interest = 13
loan = 1234
pd.DataFrame([interest,loan])

,0
0,13
1,1234
